<a href="https://colab.research.google.com/github/davidvlaminck/AWVGeneric/blob/master/UseCases/SearchAssetsWithParent/OTL_assets_met_naampad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pas eerst onderstaande parameters aan en voer het script uit. Elke keer je de parameters aanpast, moet je ook het script uitvoeren.

In [1]:
awv_acm_cookie = 'fd82962702ac4ed2a987a2adc9a95d72'

In [5]:
!sudo apt-get update -y
!sudo apt-get install python3.11

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,506 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubun

In [7]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2

!python --version

update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in auto mode
update-alternatives: using /usr/bin/python3.11 to provide /usr/bin/python3 (python3) in auto mode
Python 3.11.10


Voer onderstaande script uit om de GitHub repo te clonen.

In [2]:
! git clone https://github.com/davidvlaminck/AWVGeneric

Cloning into 'AWVGeneric'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 156 (delta 90), reused 112 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (156/156), 34.61 KiB | 581.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.


Voer onderstaande script uit om de SN Gateway te query'en

In [8]:
import sys
sys.path.insert(0, '/content/AWVGeneric')  # adds the Python files from the GitHub repo

from pathlib import Path
from prettytable import PrettyTable

from API.EMInfraClient import EMInfraClient
from API.EMInfraDomain import QueryDTO, PagingModeEnum, SelectionDTO, ExpressionDTO, TermDTO, OperatorEnum, \
    LogicalOpEnum, ExpansionsDTO, construct_naampad
from API.Enums import AuthType, Environment

eminfra_client = EMInfraClient(cookie=awv_acm_cookie, auth_type=AuthType.COOKIE, env=Environment.PRD)  # change environment here

asset_types = list(eminfra_client.get_all_otl_assettypes())
print(f'aantal OTL types: {len(asset_types)}')

type_term = TermDTO(property='type', operator=OperatorEnum.EQ, value='a7eadedf-b5cf-491b-8b89-ccced9a37004')
query_dto = QueryDTO(size=100, from_=0, pagingMode=PagingModeEnum.OFFSET,
                      expansions=ExpansionsDTO(fields=['parent']),
                      selection=SelectionDTO(
                          expressions=[ExpressionDTO(
                              terms=[type_term,
                                  TermDTO(property='actief', operator=OperatorEnum.EQ,
                                          value=True, logicalOp=LogicalOpEnum.AND),
                                  TermDTO(property='beheerobject', operator=OperatorEnum.EQ,
                                          value=None, logicalOp=LogicalOpEnum.AND, negate=True)])]))
headers = ['uuid', 'type', 'naampad', 'em_infra_link']
rows = []
for otl_asset_type in asset_types:
    print(f'querying type {otl_asset_type.korteUri}')
    type_term.value = otl_asset_type.uuid
    query_dto.from_ = 0
    rows.extend([asset.uuid, otl_asset_type.korteUri, construct_naampad(asset),
                  f'https://apps.mow.vlaanderen.be/eminfra/assets{asset.uuid}']
                for asset in eminfra_client.search_assets(query_dto))
table = PrettyTable(headers)
table.add_rows(rows)

with open('table.csv', 'w', newline='') as f_output:
    f_output.write(table.get_csv_string())


SyntaxError: f-string: unmatched '[' (EMInfraDomain.py, line 331)